In [342]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

# scale back and remmove negative values
# apply gradient dexcent attack on benign sample -> scale back + delta + modify sample by adding bytes

from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

In [343]:
data = pd.read_csv('PDFMalware2022.csv').dropna()
data = data.sample(1000)
# data.columns

# Features recognized by pdfid
pdfid_columns = ['obj', 'endobj', 'stream', 'endstream', 'xref', 'trailer', 'startxref', 'header',
                 'pages', 'isEncrypted', 'ObjStm', 'JS', 'Javascript', 'AA', 'OpenAction',
                 'Acroform', 'JBIG2Decode', 'RichMedia', 'launch', 'embedded files', 'XFA', 'Colors',
                 'Class']

#print(data['endstream'].value_counts())

In [344]:
new_labels = {'Malicious': 1, 'Benign': 0}
data['Class'] = data['Class'].map(new_labels)


new_labels = {'1(1)': '1', '2(1)': '2', '3(1)': '3', '29(2)': '29', '34(2)': '34', '2(2)': '2', '>': '0', '53(1)': '53', '5(1)': '5', '12(2)': '2', '53(2)': '53', '-1': '0', 
              '(most': '0', '_Pro_Rodeo_Pix_': '0', "_Pro_Rodeo_Pix_'": '0', 'pdfid.py': '0', 'pdfHeader)': '0', 'bytes[endHeader]': '0', 'list': '0', 'unclear': '0', 'Yes': '1', 'No': '0'}

for col in data.drop(columns=['Class']).columns:
    data[col] = data[col].replace(new_labels)

data.head()

,Fine name,pdfsize,metadata size,pages,xref Length,title characters,isEncrypted,embedded files,images,text,...,AA,OpenAction,Acroform,JBIG2Decode,RichMedia,launch,EmbeddedFile,XFA,Colors,Class
5088,7aeb9b3d775c81756b88864f3c117ae5202a452e60175c...,46.0,180.0,1.0,11.0,0.0,0.0,1.0,0,0,...,0,1,0,0,0,0,0,0,0.0,1
8928,4c0055879646ef9b1e1881deaca1e2e75fbef1121477c2...,24.0,298.0,1.0,12.0,0.0,0.0,0.0,1,0,...,0,0,0,0,0,0,0,0,0.0,0
6753,0feba1e8d7a3ab4c56ddb8e5ca75d8c0e6e663aa45872d...,80.0,229.0,20.0,76.0,0.0,0.0,0.0,1,1,...,0,0,0,0,0,0,0,0,0.0,0
1234,e73e2fc67d4c7144113079d563b5528f1071c7c068d188...,8.0,180.0,1.0,10.0,0.0,0.0,0.0,0,0,...,0,0,1,0,0,0,8,1,0.0,1
9423,8962210f3f5d0d7b4f9641d6d916bf7250480200c89458...,392.0,268.0,4.0,178.0,0.0,0.0,0.0,0,1,...,0,0,0,0,0,0,0,0,2.0,0


In [345]:
data = data[pdfid_columns]
class_data = data.drop(columns=['header', 'Class'])

x_train, x_test, y_train, y_test = train_test_split(class_data, data['Class'], test_size=0.2, random_state=77)
x_train.shape

(800, 21)

In [346]:
# Normalize features
scaler = RobustScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

pd.DataFrame(x_train, columns=class_data.columns).head()

,obj,endobj,stream,endstream,xref,trailer,startxref,pages,isEncrypted,ObjStm,...,Javascript,AA,OpenAction,Acroform,JBIG2Decode,RichMedia,launch,embedded files,XFA,Colors
0,-0.210526,-0.216216,-0.196721,-0.196721,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.157895,-0.162162,-0.065574,-0.065574,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.394737,-0.405405,-0.327869,-0.262295,-1.0,-2.0,-1.0,0.0,0.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
3,0.526316,0.540541,-0.196721,-0.196721,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.131579,-0.135135,-0.065574,-0.065574,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [347]:
def evasion_gradient_descent(x0, gradient, t, epsilon, max_iter, d_max=10):

    m = 0
    x_m = x0
    for i in range(max_iter):

        m += 1
        x_m = x_m - t * gradient
        # print(f"Iteration {i}...")
        # x_m = np.maximum(x_m, 0)
        x_m = project(x_m, x0, d_max)
        if np.linalg.norm(x_m - x0) < epsilon:
            break

    x = x_m
    return x_m


def project(x, x0, d_max):
    dist = np.linalg.norm(x - x0)
    if dist > d_max:
        return x0 + d_max * (x - x0) / dist
    return x

## SVM

In [348]:
C_values = [1, 0.8, 0.5, 0.3, 0.1]
scores = []

for c in C_values:
    svc = SVC(kernel='linear', C=c)
    svc.fit(x_train, y_train)
    y_pred = svc.predict(x_test)
    f1 = f1_score(y_test, y_pred)
    scores.append(f1)
    print(f"C = {c}, F1: {f1}")

best_c = C_values[scores.index(max(scores))]
best_c

C = 1, F1: 0.9753694581280788
C = 0.8, F1: 0.9702970297029703
C = 0.5, F1: 0.9702970297029703
C = 0.3, F1: 0.9651741293532339
C = 0.1, F1: 0.9702970297029703


1

In [349]:
svc = SVC(kernel='linear', C=1)
svc.fit(x_train, y_train)

y_pred = svc.predict(x_test)

svm_accuracy = accuracy_score(y_test, y_pred)
svm_precision = precision_score(y_test, y_pred)
svm_recall = recall_score(y_test, y_pred)
svm_f1 = f1_score(y_test, y_pred)

print(f"""SVM scores
      accuracy score: {svm_accuracy}
      precision score: {svm_precision}
      recall score: {svm_recall}
      f1 score: {svm_f1}\n""")
print(f"Confusion matrix:\n{confusion_matrix(y_test, y_pred)}")

SVM scores
      accuracy score: 0.975
      precision score: 1.0
      recall score: 0.9519230769230769
      f1 score: 0.9753694581280788

Confusion matrix:
[[96  0]
 [ 5 99]]


In [350]:
gamma_values = [10, 5, 1, 1/1000, 1/x_train.shape[0], 1/10**6]
scores = []

for g in gamma_values:
     svc_rbf = SVC(kernel='rbf', gamma=g)
     svc_rbf.fit(x_train, y_train)
     y_pred = svc_rbf.predict(x_test)
     f1 = f1_score(y_test, y_pred)
     scores.append(f1)
     print(f"Gamma = {g}, F1: {f1}")

best_gamma = gamma_values[scores.index(max(scores))]

Gamma = 10, F1: 0.8633879781420765
Gamma = 5, F1: 0.8633879781420765
Gamma = 1, F1: 0.9278350515463918
Gamma = 0.001, F1: 0.9295774647887324
Gamma = 0.00125, F1: 0.9295774647887324
Gamma = 1e-06, F1: 0.6842105263157895


In [351]:
svc_rbf = SVC(kernel='rbf', gamma=best_gamma)
svc_rbf.fit(x_train, y_train)

y_pred = svc_rbf.predict(x_test)

svm_rbf_accuracy = accuracy_score(y_test, y_pred)
svm_rbf_precision = precision_score(y_test, y_pred)
svm_rbf_recall = recall_score(y_test, y_pred)
svm_rbf_f1 = f1_score(y_test, y_pred)

print(f"""\nSVM with rbf kernel scores
      accuracy score: {svm_rbf_accuracy}
      precision score: {svm_rbf_precision}
      recall score: {svm_rbf_recall}
      f1 score: {svm_rbf_f1}\n""")
print(f"Confusion matrix:\n{confusion_matrix(y_test, y_pred)}")


SVM with rbf kernel scores
      accuracy score: 0.925
      precision score: 0.908256880733945
      recall score: 0.9519230769230769
      f1 score: 0.9295774647887324

Confusion matrix:
[[86 10]
 [ 5 99]]


In [352]:
def rbf_kernel(x, x_i, gamma=0.0001):
    return np.exp(- gamma * np.pow(np.linalg.norm(x - x_i), 2))

def poly_kernel(x, x_i, d=3, c=1):
    return np.pow((x @ x_i) + c, d)

def gradient_rbf_kernel(x, x_i, gamma):
    return -2 * gamma * np.exp(- gamma * np.pow(np.linalg.norm(x - x_i), 2)) * (x - x_i)

def gradient_poly_kernel(x, x_i, d, c):
    return d * np.pow(d * (x @ x_i + c), d - 1) * x_i


def svm_gradient(weights, feature_vector=None, support_vectors=None, kernel='linear', gamma=0.001, d=3, c=0):
    if kernel == 'linear':
        return weights
    
    delta_g = np.zeros(feature_vector.shape)
    kernel_gradient = np.zeros(feature_vector.shape)
    for i in range(support_vectors.shape[0]):
        w_i = weights[i]
        x_i = support_vectors[i]
        if kernel == 'rbf':
            kernel_gradient = gradient_rbf_kernel(feature_vector, x_i, gamma)
        elif kernel == 'poly':
            kernel_gradient = gradient_poly_kernel(feature_vector, x_i, d, c)
        # print(f"delta_g: {delta_g.shape}, weights: {weights.shape}, kernel_gradient: {kernel_gradient.shape}")
        delta_g = delta_g + w_i * kernel_gradient
    return delta_g

In [353]:
weights_linear = svc.coef_[0]
bias_linear = svc.intercept_
gradient_linear = weights_linear
print(f"Linear weights:\n{gradient_linear}")

gamma = svc_rbf.get_params()['gamma']
weights_rbf = svc_rbf.dual_coef_[0]
# print(f"{x_train_svm.iloc[svc_rbf.support_].head()}")
support_vectors_rbf = svc_rbf.support_vectors_ # x_train_svm.iloc[svc_rbf.support_].to_numpy().astype(int)
print(f"Gamma: {gamma}, Weights: {weights_rbf.shape}, Support Vectors: {support_vectors_rbf.shape}")

malicious_samples = class_data[data['Class'] == 1].iloc[:100]
x_malicious_samples = malicious_samples.to_numpy().astype(int)
modified = np.zeros(x_malicious_samples.shape)

x_malicious_samples = scaler.fit_transform(x_malicious_samples)
learning_rates = [0.001]
epsilons = [0.0001]
iterations = 200
d_values = [10, 20, 50, 100] # number of modifications
accuracy = 0

kernel = "rbf"
grad_i = 0

for lr in learning_rates:

    for d in d_values:

        modified = np.zeros(x_malicious_samples.shape)
        for i in range (len(x_malicious_samples)):
            if kernel == 'linear':
                grad_i = gradient_linear
            elif kernel == 'rbf':
                grad_i = svm_gradient(weights=weights_rbf,feature_vector=x_malicious_samples[i],support_vectors=support_vectors_rbf,kernel='rbf',gamma=gamma)

            modified[i] = evasion_gradient_descent(
                x0=x_malicious_samples[i],
                gradient=grad_i,
                t=lr,
                epsilon=epsilons[0],
                max_iter=iterations,
                d_max=d
                )
            
        modified_samples = pd.DataFrame(modified, columns=class_data.columns)
        cl = svc_rbf.predict(modified_samples)
        modified_samples['Class'] = cl
        accuracy = len(modified_samples[modified_samples['Class'] == 0]) / len(x_malicious_samples)
        print(f"Accuracy: {accuracy} with learning rat = {lr} and d_max = {d}")


modified_samples = pd.DataFrame(modified, columns=class_data.columns)
cl = svc_rbf.predict(modified_samples)
modified_samples['Class'] = cl
# print(modified_samples)

accuracy = len(modified_samples[modified_samples['Class'] == 0]) / len(x_malicious_samples)
print(f"Accuracy: {accuracy}")

print("Delta: ")
delta_df = x_malicious_samples - modified

delta_df = pd.DataFrame(delta_df, columns=class_data.columns)
delta_df

Linear weights:
[ 0.18614422 -0.88990593 -0.78421906  1.16408452 -0.56044636 -0.6941678
  0.93448403  0.00538888 -2.08004101 -0.52279466 -1.95970896  2.02236387
 -0.31625239  1.90689654 -0.12913254 -0.1381798   0.          0.39100068
  0.          2.06415785  0.04484271]
Gamma: 0.001, Weights: (439,), Support Vectors: (439, 21)


/home/paul/.local/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


Accuracy: 0.19 with learning rat = 0.001 and d_max = 10


/home/paul/.local/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


Accuracy: 0.19 with learning rat = 0.001 and d_max = 20


/home/paul/.local/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


Accuracy: 0.19 with learning rat = 0.001 and d_max = 50
Accuracy: 0.19 with learning rat = 0.001 and d_max = 100
Accuracy: 0.19
Delta: 


/home/paul/.local/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(
/home/paul/.local/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


,obj,endobj,stream,endstream,xref,trailer,startxref,pages,isEncrypted,ObjStm,...,Javascript,AA,OpenAction,Acroform,JBIG2Decode,RichMedia,launch,embedded files,XFA,Colors
0,-0.021406,-0.022459,-0.019484,-0.019507,-0.038657,-0.026484,-0.035688,-0.016773,-0.004833,-0.016652,...,0.046677,-0.004103,0.035206,-0.002196,-0.004589,0.0,0.001882,-0.002083,0.017410,0.000346
1,-0.020703,-0.021721,-0.018640,-0.018663,-0.038493,-0.026446,-0.035570,-0.017774,-0.004856,-0.016689,...,0.046076,-0.004101,0.034887,-0.002027,-0.004585,0.0,0.001891,-0.002007,0.017614,0.000080
2,-0.021462,-0.022518,-0.019487,-0.019509,-0.038665,-0.026485,-0.035695,-0.016785,-0.004839,-0.016636,...,0.046458,-0.004185,0.035287,-0.002323,-0.004585,0.0,0.001878,-0.001966,0.017329,0.000402
3,-0.021463,-0.022514,-0.019475,-0.019498,-0.038648,-0.026462,-0.035665,-0.016607,-0.004835,-0.016605,...,0.046571,-0.004136,0.035162,-0.002176,-0.004583,0.0,0.001879,-0.001957,0.017410,0.000432
4,-0.021406,-0.022459,-0.019505,-0.019528,-0.038687,-0.026501,-0.035713,-0.016797,-0.004845,-0.016653,...,0.046734,-0.004095,0.035229,-0.002191,-0.004594,0.0,0.001884,-0.001966,0.017427,0.000386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-0.021765,-0.022831,-0.019673,-0.019697,-0.038819,-0.026701,-0.035827,-0.015565,-0.004789,-0.016837,...,0.046317,-0.004448,0.035423,-0.002242,-0.004589,0.0,0.001877,-0.001872,0.017411,0.000055
96,-0.015159,-0.015712,-0.014703,-0.014710,-0.034426,-0.023253,-0.031470,-0.022037,-0.005614,-0.014351,...,0.041570,-0.004123,0.031884,-0.002398,-0.004169,0.0,0.001806,-0.002960,0.015924,0.003765
97,-0.021406,-0.022459,-0.019505,-0.019528,-0.038687,-0.026501,-0.035713,-0.016797,-0.004845,-0.016653,...,0.046734,-0.004095,0.035229,-0.002191,-0.004594,0.0,0.001884,-0.001966,0.017427,0.000386
98,-0.020851,-0.021879,-0.018824,-0.018847,-0.038545,-0.026469,-0.035623,-0.017662,-0.004856,-0.016706,...,0.046205,-0.004076,0.034955,-0.002043,-0.004590,0.0,0.001890,-0.002002,0.017610,0.000129


In [354]:
scaled_back = scaler.inverse_transform(modified_samples.drop(columns=['Class']))

modified_df = pd.DataFrame(scaled_back, columns=class_data.columns).astype(int)
modified_df['Class'] = modified_samples['Class']

modified_df

,obj,endobj,stream,endstream,xref,trailer,startxref,pages,isEncrypted,ObjStm,...,AA,OpenAction,Acroform,JBIG2Decode,RichMedia,launch,embedded files,XFA,Colors,Class
0,10,10,3,3,1,1,1,1,0,0,...,0,0,0,0,0,0,1,0,0,1
1,21,21,10,10,1,1,1,1,0,0,...,0,0,1,0,0,0,0,0,0,0
2,11,11,2,2,1,1,1,1,0,0,...,0,0,1,0,0,0,0,0,0,1
3,9,9,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,9,9,3,3,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,12,12,5,5,1,1,1,3,0,0,...,0,0,0,0,0,0,0,0,0,1
96,60,60,16,16,2,2,2,0,0,1,...,0,0,0,0,0,0,0,0,3,0
97,9,9,3,3,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
98,19,19,9,9,1,1,1,1,0,0,...,0,0,1,0,0,0,0,0,0,0


In [355]:
modified_df.describe()

,obj,endobj,stream,endstream,xref,trailer,startxref,pages,isEncrypted,ObjStm,...,AA,OpenAction,Acroform,JBIG2Decode,RichMedia,launch,embedded files,XFA,Colors,Class
count,100.000000,100.000000,100.0000,100.0000,100.00000,100.000000,100.000000,100.000000,100.0,100.000000,...,100.000000,100.0,100.000000,100.0,100.0,100.0,100.000000,100.0,100.000000,100.000000
mean,17.790000,17.300000,6.2300,6.2300,0.85000,0.950000,0.890000,1.760000,0.0,0.160000,...,0.040000,0.0,0.240000,0.0,0.0,0.0,0.040000,0.0,1.430000,0.810000
std,36.648837,36.402214,14.5622,14.5622,0.51981,0.479372,0.510397,4.605925,0.0,0.787786,...,0.196946,0.0,0.429235,0.0,0.0,0.0,0.196946,0.0,11.946222,0.394277
min,0.000000,0.000000,0.0000,0.0000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,-1.000000,0.000000
25%,8.000000,8.000000,1.0000,1.0000,1.00000,1.000000,1.000000,1.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,1.000000
50%,9.000000,9.000000,3.0000,3.0000,1.00000,1.000000,1.000000,1.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,1.000000
75%,18.250000,17.250000,6.0000,6.0000,1.00000,1.000000,1.000000,1.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,1.000000
max,346.000000,346.000000,123.0000,123.0000,2.00000,2.000000,3.000000,36.000000,0.0,6.000000,...,1.000000,0.0,1.000000,0.0,0.0,0.0,1.000000,0.0,115.000000,1.000000


In [356]:
delta_scaled = scaler.inverse_transform(delta_df).astype(int)

delta_df_scaled = pd.DataFrame(delta_scaled, columns=delta_df.columns)
delta_df_scaled

,obj,endobj,stream,endstream,xref,trailer,startxref,pages,isEncrypted,ObjStm,...,Javascript,AA,OpenAction,Acroform,JBIG2Decode,RichMedia,launch,embedded files,XFA,Colors
0,8,8,2,2,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,8,8,2,2,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,8,8,2,2,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,8,8,2,2,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,8,8,2,2,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,8,8,2,2,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
96,8,8,2,2,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
97,8,8,2,2,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
98,8,8,2,2,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [357]:
malicious_sample = class_data[data['Class'] == 1].sample(n=1).astype(int)
#malicious_mod = malicious_sample - delta_df_scaled.iloc[malicious_sample.index]

#pred = svc.predict(malicious_mod)
#print(f"Predicted class: {pred}")
#malicious_mod

## MLP

In [358]:
x_train_mlp, x_test_mlp, y_train_mlp, y_test_mlp = train_test_split(class_data, data['Class'], test_size=0.2, random_state=77)

In [359]:
# Normalize features
normalizer = Normalizer()
x_train_mlp = normalizer.fit_transform(x_train_mlp)
x_test_mlp = normalizer.transform(x_test_mlp)

pd.DataFrame(x_train_mlp, columns=class_data.columns).head()

,obj,endobj,stream,endstream,xref,trailer,startxref,pages,isEncrypted,ObjStm,...,Javascript,AA,OpenAction,Acroform,JBIG2Decode,RichMedia,launch,embedded files,XFA,Colors
0,0.673575,0.673575,0.096225,0.096225,0.096225,0.096225,0.096225,0.096225,0.0,0.000000,...,0.096225,0.096225,0.096225,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.658145,0.658145,0.219382,0.219382,0.073127,0.073127,0.073127,0.073127,0.0,0.000000,...,0.073127,0.000000,0.073127,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.000000,0.000000,-0.447214,0.000000,0.000000,-0.447214,0.000000,0.447214,0.0,-0.447214,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,-0.447214
3,0.706243,0.706243,0.020178,0.020178,0.020178,0.020178,0.020178,0.020178,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.666667,0.666667,0.200000,0.200000,0.066667,0.066667,0.066667,0.066667,0.0,0.000000,...,0.066667,0.000000,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [360]:
hidden_layers = (128,)
activation = 'tanh'
solver = 'adam'

mlp = MLPClassifier(hidden_layer_sizes=hidden_layers, activation=activation, solver=solver)
mlp.fit(x_train_mlp, y_train_mlp)

y_pred = mlp.predict(x_test_mlp)

mlp_accuracy = accuracy_score(y_test_mlp, y_pred)
mlp_precision = precision_score(y_test_mlp, y_pred)
mlp_recall = recall_score(y_test_mlp, y_pred)
mlp_f1 = f1_score(y_test_mlp, y_pred)

print(f"""MLP scores
      accuracy score: {mlp_accuracy}
      precision score: {mlp_precision}
      recall score: {mlp_recall}
      f1 score: {mlp_recall}\n""")
print(f"Confusion matrix:\n{confusion_matrix(y_test_mlp, y_pred)}")



MLP scores
      accuracy score: 0.965
      precision score: 0.98989898989899
      recall score: 0.9423076923076923
      f1 score: 0.9423076923076923

Confusion matrix:
[[95  1]
 [ 6 98]]


/home/paul/.local/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [361]:
def tanh(z):
    return np.tanh(z)

def sigmoid(z): 
    return 1 / (1 + np.exp(-z))

def relu(z):
    return max(0, z)

def tanh_derivative(z):
    return 1 - np.pow(tanh(z), 2)

def sigmoid_derivative(z):
    return 1 - sigmoid(z)

def relu_derivative(z):
    return z>=0


def mlp_discriminant(x, hidden_weights, hidden_bias, output_weights, output_bias):
    g = tanh(output_weights.T @ tanh(hidden_weights.T @ x + hidden_bias) + output_bias)
    return g


def mlp_gradient(x, hidden_weights, hidden_bias, output_weights, output_bias):
    g = mlp_discriminant(x, hidden_weights, hidden_bias, output_weights, output_bias)
    delta_k = tanh(hidden_weights.T @ x + hidden_bias)

    delta_g = np.zeros(x.shape)
    for i in range(x.shape[0]):
        # delta_g[i] = g * (1 - np.pow(g, 1)) * (output_weights.T @ (delta_k * (1 - np.pow(delta_k, 1)) * hidden_weights[i]))
        delta_g[i] = g * (1 - np.pow(g, 2)) * (output_weights.T @ (delta_k * (1 - np.pow(delta_k, 2)) * hidden_weights[i]))

    return delta_g

weights = mlp.coefs_

bias = mlp.intercepts_
output_weights = np.array(weights[-1])
output_bias = np.array(bias[-1])
hidden_weights = np.array(weights[0])
hidden_bias = np.array(bias[0])

In [362]:
malicious_samples = class_data[data['Class'] == 1]
x_malicious_samples = malicious_samples.to_numpy().astype(int)
modified = np.zeros(x_malicious_samples.shape)

norms = np.linalg.norm(x_malicious_samples, axis=1, keepdims=True)
mean_norms = np.mean(norms)
x_malicious_samples = normalizer.fit_transform(x_malicious_samples)

for i in range (len(x_malicious_samples)):
    mlp_gr = mlp_gradient(x_malicious_samples[i], hidden_weights, hidden_bias, output_weights, output_bias)
    modified[i] = evasion_gradient_descent(
        x0=x_malicious_samples[i],
        gradient=mlp_gr,
        t=0.5,
        epsilon=0.00001,
        max_iter=200
        )

modified_samples = pd.DataFrame(modified, columns=class_data.columns)
cl = mlp.predict(modified_samples)
modified_samples['Class'] = cl
modified_samples.describe()

accuracy = len(modified_samples[modified_samples['Class'] == 0]) / len(x_malicious_samples)
print(f"Accuracy: {accuracy}")

print("Delta: ")
delta_df = x_malicious_samples - modified

delta_df = pd.DataFrame(delta_df, columns=class_data.columns)
delta_df#.iloc[0]

/tmp/ipykernel_1061617/1999807400.py:32: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  delta_g[i] = g * (1 - np.pow(g, 2)) * (output_weights.T @ (delta_k * (1 - np.pow(delta_k, 2)) * hidden_weights[i]))


Accuracy: 0.25749559082892415
Delta: 


/home/paul/.local/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(


,obj,endobj,stream,endstream,xref,trailer,startxref,pages,isEncrypted,ObjStm,...,Javascript,AA,OpenAction,Acroform,JBIG2Decode,RichMedia,launch,embedded files,XFA,Colors
0,-4.762765e-02,4.688667e-02,-3.391287e-02,-1.637600e-01,-1.197357e-01,1.324586e-01,2.166370e-02,-1.205112e-01,2.260818e-02,-8.573828e-02,...,6.098057e-02,1.215284e-01,8.805368e-02,7.254957e-02,5.742668e-04,-4.320691e-02,-3.427306e-02,9.543488e-04,1.453665e-01,1.684620e-01
1,-4.983406e-01,-2.996093e-02,-1.037525e+00,-3.173228e+00,-1.428378e+00,2.406056e+00,6.100349e-01,-1.504371e+00,4.390955e-01,3.054052e-01,...,6.661490e-01,3.400654e+00,1.563019e-01,-8.870445e-01,-1.029712e+00,-1.997696e+00,-2.043409e-01,-1.152072e+00,1.281823e+00,3.509449e+00
2,-3.095726e-08,4.205571e-08,-4.828663e-09,-7.421779e-08,-8.656144e-08,7.544542e-08,1.556056e-08,-5.209139e-08,1.165321e-08,-1.017513e-07,...,4.972736e-08,2.412193e-08,7.704178e-08,1.089779e-07,-1.261678e-08,1.557205e-08,-2.339755e-08,1.356309e-08,1.167272e-07,9.171718e-08
3,-5.133822e-07,6.698311e-07,-4.339672e-08,-1.440981e-06,-1.375219e-06,1.292158e-06,1.063790e-07,-1.094567e-06,1.939254e-07,-1.392545e-06,...,6.431048e-07,7.659138e-07,1.185885e-06,1.455197e-06,1.481694e-07,-4.668718e-08,-3.941631e-07,3.953523e-07,1.733690e-06,1.393291e-06
4,-7.464608e-03,8.170451e-03,-3.556712e-03,-2.050708e-02,-1.530672e-02,1.650278e-02,1.285678e-03,-1.472064e-02,5.329867e-03,-1.250537e-02,...,3.539081e-03,1.612167e-02,4.343071e-03,8.809324e-03,4.158598e-03,-7.868916e-03,-3.049162e-03,4.150148e-03,1.115388e-02,2.042444e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,3.782663e-02,1.099900e-01,2.062894e-01,6.461467e-01,-4.470374e-01,-2.746276e-01,1.378325e-01,3.272204e-02,-4.646259e-01,-1.011603e+00,...,1.019273e+00,-1.371039e+00,1.493058e+00,1.947631e+00,-3.596474e-01,1.336837e+00,-4.503594e-01,2.750837e-01,2.032394e+00,-3.244061e-01
563,-1.553502e-12,1.006233e-12,-3.320677e-13,-7.626122e-13,-1.193754e-12,7.772671e-13,3.524958e-13,-6.080136e-13,9.360445e-13,-2.567507e-12,...,8.944512e-13,-2.868191e-14,5.786827e-13,2.873248e-12,5.590089e-13,-4.873108e-14,-2.034970e-13,8.352079e-13,3.751870e-13,1.794818e-12
564,-5.198483e-08,6.993469e-08,1.181531e-08,-1.281903e-07,-1.353117e-07,1.312478e-07,1.561168e-08,-1.022734e-07,4.298158e-08,-1.475027e-07,...,6.523797e-08,9.608876e-08,6.278546e-08,1.452158e-07,4.250071e-08,-5.342394e-09,-3.434012e-08,4.557338e-08,1.294409e-07,1.364629e-07
565,-3.391560e-02,6.683158e-01,1.959576e-01,-1.777152e+00,-1.189055e+00,1.510915e+00,-4.864817e-01,-1.106356e+00,4.179012e-01,-1.371086e-02,...,3.120927e-01,2.388409e+00,-2.415978e-01,-8.070211e-01,8.824243e-01,-1.317379e+00,-4.810004e-01,-1.157651e-02,2.668704e-01,8.358910e-01


In [363]:
scaled_back = modified_samples.drop(columns=['Class']) * norms

modified_df = pd.DataFrame(scaled_back, columns=class_data.columns).astype(int)
modified_df['Class'] = modified_samples['Class']


# scaled_back = normalizer.inverse_transform(modified_samples.drop(columns=['Class']))

# modified_df = pd.DataFrame(scaled_back, columns=class_data.columns).astype(int)
# modified_df['Class'] = modified_samples['Class']

modified_df

,obj,endobj,stream,endstream,xref,trailer,startxref,pages,isEncrypted,ObjStm,...,AA,OpenAction,Acroform,JBIG2Decode,RichMedia,launch,embedded files,XFA,Colors,Class
0,10,9,3,5,2,0,0,2,0,1,...,-1,0,-1,0,0,0,0,-2,-2,0
1,37,21,44,114,48,-78,-19,50,-14,-10,...,-112,-5,30,33,65,6,38,-41,-115,0
2,11,10,2,2,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,9,8,1,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,9,8,3,3,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,55,47,19,-25,46,28,-8,-2,47,111,...,138,-151,-196,36,-135,45,-27,-205,32,0
563,0,0,1,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
564,13,12,1,2,1,0,0,6,0,0,...,0,0,0,0,0,0,0,0,0,1
565,5,0,0,14,10,-10,3,9,-3,0,...,-17,1,6,-6,10,3,0,-2,-6,1


In [364]:
delta_scaled = (delta_df * norms).astype(int)

delta_scaled

# delta_scaled = scaler.inverse_transform(delta_df).astype(int)

# delta_df_scaled = pd.DataFrame(delta_scaled, columns=delta_df.columns)
# delta_df_scaled

,obj,endobj,stream,endstream,xref,trailer,startxref,pages,isEncrypted,ObjStm,...,Javascript,AA,OpenAction,Acroform,JBIG2Decode,RichMedia,launch,embedded files,XFA,Colors
0,0,0,0,-2,-1,1,0,-1,0,-1,...,0,1,1,1,0,0,0,0,2,2
1,-16,0,-34,-104,-47,79,20,-49,14,10,...,21,112,5,-29,-33,-65,-6,-38,42,115
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,3,11,20,65,-45,-27,13,3,-47,-102,...,103,-138,151,197,-36,135,-45,27,205,-32
563,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
564,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
565,0,5,1,-13,-9,11,-3,-8,3,0,...,2,18,-1,-6,6,-10,-3,0,2,6


In [365]:
malicious_sample = class_data[data['Class'] == 1].sample(n=1).astype(int)
malicious_mod = malicious_sample - delta_scaled[malicious_sample.index]

pred = svc.predict(malicious_mod)
print(f"Predicted class: {pred}")
malicious_mod

KeyError: "None of [Index([2195], dtype='int64')] are in the [columns]"